# **VR booster_coverage_archived_20220904**


# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/utils/conversions.ipynb",
        "src/utils/protos.ipynb"
    ]
}
```

TABLE

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM SYS.TABLES WHERE object_id = OBJECT_ID('VWSARCHIVE.RIVM_VACCINATIONS_BOOSTER_SHOTS_COVERAGE_VR_ARCHIVED_20220904'))
BEGIN
    CREATE TABLE [VWSARCHIVE].[RIVM_VACCINATIONS_BOOSTER_SHOTS_COVERAGE_VR_ARCHIVED_20220904]
    (
        [ID] [INT] PRIMARY KEY IDENTITY(1,1),
        [DATE_LAST_INSERTED] [DATETIME] NOT NULL DEFAULT GETDATE(),
        [VERSION] [INT] NULL,
        [DATE_OF_REPORT] [DATETIME] NULL,
        [DATE_OF_STATISTICS] [DATETIME] NULL,
        [POPULATION] [BIGINT] NULL,
        [REGION_CODE] [VARCHAR] (256) NULL,
        [REGION_LEVEL] [VARCHAR] (256) NULL,
        [REGION_NAME] [VARCHAR] (256) NULL,
        [AGE_GROUP] [VARCHAR] (256) NULL,
        [VACCINATION_SERIE] [VARCHAR] (256) NULL,
        [PERCENTAGE] [NUMERIC](10,2) NULL,
        [PERCENTAGE_LABEL] [VARCHAR] (256) NULL  
    );
END
GO

STORED PROCEDURE

In [ ]:

CREATE OR ALTER PROCEDURE [dbo].[SP_RIVM_VACCINATIONS_BOOSTER_SHOTS_COVERAGE_VR_ARCHIVED_20220904] AS
BEGIN
    TRUNCATE TABLE [VWSARCHIVE].[RIVM_VACCINATIONS_BOOSTER_SHOTS_COVERAGE_VR_ARCHIVED_20220904]

    INSERT INTO [VWSARCHIVE].[RIVM_VACCINATIONS_BOOSTER_SHOTS_COVERAGE_VR_ARCHIVED_20220904] (
        [DATE_LAST_INSERTED],
        [VERSION],
        [DATE_OF_REPORT],
        [DATE_OF_STATISTICS],
        [POPULATION],
        [REGION_CODE],
        [REGION_LEVEL],
        [REGION_NAME],
        [AGE_GROUP],
        [VACCINATION_SERIE],
        [PERCENTAGE],
        [PERCENTAGE_LABEL]
    )
    SELECT
        [DATE_LAST_INSERTED],
        [VERSION],
        [DATE_OF_REPORT],
        [DATE_OF_STATISTICS],
        [POPULATION],
        [REGION_CODE],
        [REGION_LEVEL],
        [REGION_NAME],
        [AGE_GROUP],
        [VACCINATION_SERIE],
        [PERCENTAGE],
        [PERCENTAGE_LABEL]
    FROM [VWSDEST].[RIVM_VACCINATIONS_BOOSTER_SHOTS_COVERAGE_VR]
    WHERE [DATE_LAST_INSERTED] >= '2022-09-07' AND [DATE_LAST_INSERTED] < '2022-09-08' -- get first insertion after 20220904
END


VIEW

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

--1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_VACCINATIONS_BOOSTER_SHOTS_COVERAGE_VR_ARCHIVED_20220904]
AS
    WITH CTE AS (
        SELECT
            [REGION_CODE] AS [VRCODE],
            [AGE_GROUP],
            UPPER(DATENAME(dw, [DATE_OF_STATISTICS])) AS [DAY_OF_WEEK],
            [DBO].[CONVERT_DATETIME_TO_UNIX]([DATE_OF_STATISTICS]) AS [DATE_UNIX],            
            [DBO].[CONVERT_DATETIME_TO_UNIX]([DATE_LAST_INSERTED]) AS [DATE_OF_INSERTION_UNIX],
            [PERCENTAGE],
            [PERCENTAGE_LABEL]
        FROM [VWSARCHIVE].[RIVM_VACCINATIONS_BOOSTER_SHOTS_COVERAGE_VR_ARCHIVED_20220904] WITH(NOLOCK)
        WHERE [REGION_CODE] LIKE 'VR%'
            AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM  [VWSARCHIVE].[RIVM_VACCINATIONS_BOOSTER_SHOTS_COVERAGE_VR_ARCHIVED_20220904] WITH(NOLOCK))
            AND [AGE_GROUP] IN  ('18+', '12+')
            AND UPPER([VACCINATION_SERIE]) IN ('BOOSTER1_COVERAGE', 'PRIMARY_SERIES_PLUS_1', '')

    )
    SELECT 
        T2.[VRCODE],
        [AGE_GROUP],
        [PERCENTAGE], 
        [PERCENTAGE_LABEL],
        [DATE_OF_INSERTION_UNIX],
        T2.[DATE_UNIX]
    FROM CTE T1
    INNER JOIN (
        SELECT
            MAX([DATE_UNIX]) AS [DATE_UNIX],
            [VRCODE]
        FROM CTE
        -- WHERE [DAY_OF_WEEK] = 'SUNDAY'
        GROUP BY [VRCODE]
    ) AS T2 ON T1.[VRCODE] = T2.[VRCODE] AND T1.[DATE_UNIX] = T2.[DATE_UNIX]
GO

PROTOS

In [ ]:
-- -- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- -- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

DECLARE @current_proto_name VARCHAR(50)

DECLARE @proto_name_table TABLE (
    [PROTO_NAME] VARCHAR(50)
);

-- 1) CREATE PROTO(S)...
INSERT INTO @proto_name_table 
SELECT Distinct [NAME]
  FROM [DATATINO_PROTO_1].[PROTOS]
  WHERE NAME NOT LIKE 'VR_COLLECTION' AND NAME LIKE 'VR%' 
ORDER BY NAME

DECLARE CUR CURSOR LOCAL FAST_FORWARD
FOR
SELECT [PROTO_NAME]
FROM @proto_name_table;

OPEN CUR;

FETCH NEXT FROM CUR
INTO @current_proto_name;

WHILE @@FETCH_STATUS = 0
BEGIN

-- 1) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_RIVM_VACCINATIONS_BOOSTER_SHOTS_COVERAGE_VR_ARCHIVED_20220904',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'booster_coverage_archived_20220904',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = @current_proto_name,
        @constraint_key_name VARCHAR(50)= 'VRCODE',
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = @current_proto_name,
        @columns VARCHAR(256) = 'AGE_GROUP|PERCENTAGE|PERCENTAGE_LABEL|DATE_OF_INSERTION_UNIX|DATE_UNIX',
        @layout_type_id VARCHAR(50) = '2',
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

-- 2) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 3) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

FETCH NEXT FROM CUR
    INTO @current_proto_name;    
END

CLOSE CUR;
DEALLOCATE CUR;
DELETE FROM @proto_name_table;
GO